In [1]:
import json
from tqdm import tqdm

In [2]:
big = "memoryalpha"
small = "stexpanded"
top_k = 10

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/_matched_nodes_communities_leiden/top_10_pairs/" + small + "-" + big + "_top_10_pairs.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"

big_communities = []
gold_pairs = []
matched_communities = []

In [3]:
with open(communities_big_file) as cbf, open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf:

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [4]:
with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

In [5]:
matched_communities

[[0, [942, 1050, 1051, 935, 938, 57, 1054, 1154, 2023, 923]],
 [1, [572, 811, 685, 777, 694, 3, 29, 742, 33, 746]],
 [2, [445, 1174, 2211, 164, 2172, 1050, 1229, 1135, 925, 598]],
 [4, [0, 27, 367, 269, 1203, 545, 1725, 157, 1732, 268]],
 [6, [1650, 1629, 2096, 1665, 1631, 1674, 911, 912, 1160, 1673]],
 [7, [1937, 1019, 13, 1451, 858, 2135, 869, 2146, 1871, 822]],
 [8, [2194, 1160, 911, 2245, 1728, 636, 1163, 1650, 1174, 1137]],
 [9, [918, 923, 2023, 1637, 925, 1051, 1050, 448, 451, 942]],
 [10, [1019, 1937, 13, 2135, 858, 1451, 3, 2146, 869, 822]],
 [12, [1051, 97, 2211, 1050, 450, 1054, 603, 649, 935, 498]],
 [13, [422, 2234, 359, 360, 361, 824, 2038, 458, 863, 457]],
 [14, [1154, 86, 935, 879, 444, 1244, 1050, 2051, 427, 85]],
 [15, [1037, 1805, 48, 1803, 1220, 1052, 85, 441, 1740, 69]],
 [16, [2024, 870, 923, 2208, 445, 2023, 1146, 1039, 2209, 925]],
 [17, [800, 41, 806, 777, 29, 726, 33, 817, 748, 706]],
 [19, [910, 912, 1665, 911, 877, 915, 481, 2306, 2307, 1650]],
 [20, [27, 592

In [6]:
found = 0
# all_pairs = len(gold_pairs)
all_pairs = len(gold_not_exact)

# for gold_pair in tqdm(gold_pairs):
for gold_pair in tqdm(gold_not_exact):
    for matched_community in matched_communities:
        node = matched_community[0]
        community = set()
        for i in range(0, top_k):
            community.update(big_communities[matched_community[1][i]])
        if gold_pair[1] == node and gold_pair[0] in community:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 148/148 [00:40<00:00,  3.66it/s]

Gold pairs in matched communities top 10: 58.78378378378378%


In [7]:
found_big = 0
for gold_pair in gold_not_exact:
    for big_community in big_communities:
        if gold_pair[0] in big_community:
            found_big += 1
            break

print("Big: " + str(found_big / all_pairs * 100) + "%")

Big: 100.0%
